In [1]:
import pandas as pd 
import numpy as np
df = pd.read_csv("titanic.csv")
df = df.drop(columns=["PassengerId","Name","Ticket","Cabin"])

#Train-Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=["Survived"]),df["Survived"],test_size=0.2,random_state=42)

#Missing value imputation
import numpy as np
from sklearn.impute import SimpleImputer
si_age = SimpleImputer()
si_embarked=SimpleImputer(strategy="most_frequent")

#Age 
X_train_age_imputed = si_age.fit_transform(X_train[["Age"]])
X_test_age_imputed = si_age.transform(X_test[["Age"]])

#Embarked 
X_train_embarked_imputed = si_embarked.fit_transform(X_train[["Embarked"]])
X_test_embarked_imputed = si_embarked.transform(X_test[["Embarked"]])

#OneHotEncoding for Sex and Embarked
from sklearn.preprocessing import OneHotEncoder
ohe_sex = OneHotEncoder(handle_unknown="ignore",sparse_output=False)
ohe_em = OneHotEncoder(handle_unknown="ignore",sparse_output=False)

#Sex encoding
X_train_sex_encoded = ohe_sex.fit_transform(X_train[["Sex"]])
X_test_sex_encoded = ohe_sex.transform(X_test[["Sex"]])

#Embarked encoding
X_train_embarked_encoded = ohe_em.fit_transform(X_train_embarked_imputed)
X_test_embarked_encoded = ohe_em.transform(X_test_embarked_imputed)

#Concatenations
#train
X_train_rem = X_train.drop(columns=["Sex","Embarked","Age"])
X_train = np.concatenate((X_train_rem,X_train_age_imputed,X_train_sex_encoded,X_train_embarked_encoded),axis=1)
#test
X_test_rem = X_test.drop(columns=["Sex","Embarked","Age"])
X_test = np.concatenate((X_test_rem,X_test_age_imputed,X_test_sex_encoded,X_test_embarked_encoded),axis=1)

#Modelling 
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
#Training
clf.fit(X_train,y_train)
#Prediction
pred = clf.predict(X_test)
#Testing
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,pred)


array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1], dtype=int64)

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense

model = Sequential()

conv_base = VGG16(weights="imagenet",include_top=False,input_shape=(256,256,3))
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

conv_base.trainable = False


model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

input_1 = Input(shape=(32,))
hidden1 = Dense(8,activation="relu")(input_1)
hidden2 = Dense(4,activation="relu")(hidden1)

input_2 = Input(shape=(128,))
hidden3 = Dense(64,activation="relu")(input_2)
hidden4 = Dense(32,activation="relu")(hidden3)
hidden5 = Dense(4,activation="relu")(hidden4)

concatenate1 = Concatenate([hidden2,hidden5])

hidden6 = Dense(2,activation="relu")(concatenate1)
output1 = Dense(1,activation="linear")(hidden6)

model = Model(inputs=[input_1,input_2],outputs=output1)

from keras.utils import plot_model
plot_model(model,show_shapes=True)

from keras.datasets import imdb
from keras import Sequential
from keras.layers import Dense,SimpleRNN,Embedding,Flatten
from keras.utils import pad_sequences

(X_train,y_train),(X_test,y_test) = imdb.load_data()

X_train = pad_sequences(X_train,padding='post')
X_test = pad_sequences(X_test,padding='post')

length = len(X_train[0])

model = Sequential()

model.add(SimpleRNN(32,input_shape=(length,1),return_sequences=False))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))